In [14]:
##RandomizedSearchCV - HYPER TUNNING

In [18]:
import sklearn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.constraints import maxnorm
from keras.layers import Dropout

from keras.wrappers.scikit_learn import KerasClassifier

def create_model(optimizer='adam',dropout_rate=0.0, weight_constraint=0,neurons=1,activation='relu'):
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation, kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(neurons, kernel_initializer='uniform', activation='sigmoid'))
	model.add(Dropout(dropout_rate))

	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))

	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model



seed = 7
numpy.random.seed(seed)

dataset = numpy.loadtxt("/content/drive/My Drive/DATASETS/pima-indians-diabetes.csv", delimiter=",")

X = dataset[:,0:8]
Y = dataset[:,8]

model = KerasClassifier(build_fn=create_model, verbose=0,batch_size=100,epochs=20)


neurons=[1,2,3,4,5,6,7]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

param_grid = dict(neurons=neurons,optimizer=optimizer,activation=activation)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=KFold(5),verbose=3)

grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   34.5s finished


Best: 0.651159 using {'optimizer': 'SGD', 'neurons': 7, 'activation': 'tanh'}
0.651159 (0.052445) with: {'optimizer': 'SGD', 'neurons': 7, 'activation': 'tanh'}
0.651159 (0.052445) with: {'optimizer': 'Adam', 'neurons': 4, 'activation': 'sigmoid'}
0.493379 (0.159861) with: {'optimizer': 'Adadelta', 'neurons': 3, 'activation': 'tanh'}
0.651159 (0.052445) with: {'optimizer': 'SGD', 'neurons': 3, 'activation': 'linear'}
0.651159 (0.052445) with: {'optimizer': 'RMSprop', 'neurons': 2, 'activation': 'softplus'}
0.651159 (0.052445) with: {'optimizer': 'Adamax', 'neurons': 6, 'activation': 'linear'}
0.599211 (0.125526) with: {'optimizer': 'Adagrad', 'neurons': 5, 'activation': 'softmax'}
0.651159 (0.052445) with: {'optimizer': 'Adam', 'neurons': 2, 'activation': 'softplus'}
0.599211 (0.125526) with: {'optimizer': 'Adadelta', 'neurons': 1, 'activation': 'softsign'}
0.651159 (0.052445) with: {'optimizer': 'SGD', 'neurons': 1, 'activation': 'relu'}
